In [1]:
import cognitive_face as cf

key = '41c640b5a5ed45699e0797a76c7af4be'
cf.Key.set(key)
base_url = 'https://westus.api.cognitive.microsoft.com/face/v1.0/'
cf.BaseUrl.set(base_url)

In [2]:
group_lists = cf.person_group.lists()
for group in group_lists:
    group_id = group['personGroupId']
    print(group_id)
    if all(dic['name'] != 'Dummy' for dic in cf.person.lists('my_friends')):
        responses = cf.person.create('my_friends', 'Dummy')
        print('Added {}'.format(responses['personId']))
    else:
        print('Dummy already exists')
    responses = cf.person.lists('my_friends')
    print(responses)
#     group_status = cf.person_group.get_status(group_id)
#     print(group_status)
#     person_lists = 

my_friends
Dummy already exists
[{'personId': '7e47ec20-8619-4d04-8368-b73ef7b66245', 'persistedFaceIds': [], 'name': 'Dummy', 'userData': None}, {'personId': '7fb4d34d-a8fc-41f9-a104-f97a137c9416', 'persistedFaceIds': ['0012fd92-f7e5-42e9-bfde-a1899036daf2', '161cd41a-e331-4ca2-a527-75d37613ffd9', '1d4d3b66-bec6-42f4-93b3-60d9efba6e15', '4f026aa6-3ab7-4626-8a29-f48a13065d8f', '55d08301-4e71-42e0-96da-24ade9882e4a', '7038f4d9-dc18-41a9-81ea-b0cf10076c09', '89000ce9-1943-42e7-8524-f11f4445c28f', '95924d55-cdf3-4f52-a163-36e50a894aad', '9df1809d-1533-4c92-9a80-f20f331408d2', '9e6e95f7-6df5-4f82-ad6c-f04376fb1b62', 'ec411cbe-cfed-45c1-b694-a799f44a39ab', 'ec7d5a88-a65f-41f5-9def-5a3e41d5feb5', 'f3e8258c-d2cb-4b78-8bb3-a004502158cd'], 'name': 'Una', 'userData': None}, {'personId': 'a491a4f5-fe17-4f4b-b60f-c5d5f1bc751b', 'persistedFaceIds': ['08e00875-0e7e-43e2-b1c7-943b2503efd5', '0c057984-dd25-4bc6-9e2a-c3a9ddb95fa6', '1c08ae0a-b1eb-470f-b339-f4456464b435', '207aac54-77fe-497e-aad8-54e3be

In [7]:
from camera_pi import Camera
from io import BytesIO
from PIL import Image
from time import sleep
from datetime import datetime

camera = Camera()

image = '/home/pi/sherlock-azure/faces/Detection_%s.png' %(datetime.now().strftime('%Y%m%d_%H%M%S'))
with open(image, 'wb') as f:
    f.write(camera.get_frame())
    f.close()
print('Image captured. Waiting for detection results...')
detect_responses = cf.face.detect(image)
if len(detect_responses) == 0:
    print('I see no one')
else:
    face_id_list = []
    for faces in detect_responses:
        face_id = faces['faceId']
        print(face_id)
        face_id_list.append(face_id)
    identify_responses = cf.face.identify(face_id_list, 'my_friends')
    for person in cf.person.lists('my_friends'):
        identified = identify_responses[0]['candidates'][0]
        if person['personId'] == identified['personId']:
            responses = cf.person.add_face(image, 'my_friends', person['personId'])
            responses = cf.person_group.train('my_friends')
            identified_name = person['name']
            print('I see ' + identified_name)
            break

Image captured. Waiting for detection results...
1f131c4f-7c26-4f88-9b55-50d8d3ac1a09
I see Haichun


In [6]:
name = 'Haichun'
person_list = cf.person.lists('my_friends')
if all(dic['name'] != name for dic in person_list):
    responses = cf.person.create('my_friends', name)
    person_id = responses['personId']
    print('Added {}'.format(person_id))
else:
    for dic in person_list:
        if dic['name'] == name:
            person_id = dic['personId']
            break
for num in range(5):
    image = '/home/pi/sherlock-azure/faces/%s_%d.png' %(name, num+1)
    with open(image, 'wb') as f:
        f.write(camera.get_frame())
        f.close()
    sleep(0.1)
    print('Image Captured %d/5' %num)
for num in range(5):
    image = '/home/pi/sherlock-azure/faces/%s_%d.png' %(name, num+1)
    responses = cf.person.add_face(image, 'my_friends', person_id)
    print('Adding Image %d..%s' %(num, responses))
responses = cf.person.lists('my_friends')
print('Get list ..%s' %(responses))
responses = cf.person_group.train('my_friends')
print('Training %s' %(responses))

Image Captured 0/5
Image Captured 1/5
Image Captured 2/5
Image Captured 3/5
Image Captured 4/5
Adding Image 0..{'persistedFaceId': '261aee17-e998-42e7-aa35-fd532f8238d7'}
Adding Image 1..{'persistedFaceId': 'a0ba193b-ae99-4598-825d-87545760d426'}
Adding Image 2..{'persistedFaceId': 'c3a12033-3bf6-4c33-ace9-572bc4312044'}
Adding Image 3..{'persistedFaceId': '41ccc680-46fc-4ef9-ad6c-6cf1cb41ac15'}
Adding Image 4..{'persistedFaceId': 'f6247ac2-b57b-46a9-806f-c68142f20bd6'}
Get list ..[{'personId': '7e47ec20-8619-4d04-8368-b73ef7b66245', 'persistedFaceIds': [], 'name': 'Dummy', 'userData': None}, {'personId': '7fb4d34d-a8fc-41f9-a104-f97a137c9416', 'persistedFaceIds': ['0012fd92-f7e5-42e9-bfde-a1899036daf2', '161cd41a-e331-4ca2-a527-75d37613ffd9', '1d4d3b66-bec6-42f4-93b3-60d9efba6e15', '4f026aa6-3ab7-4626-8a29-f48a13065d8f', '55d08301-4e71-42e0-96da-24ade9882e4a', '7038f4d9-dc18-41a9-81ea-b0cf10076c09', '89000ce9-1943-42e7-8524-f11f4445c28f', '95924d55-cdf3-4f52-a163-36e50a894aad', '9df18

In [1]:
from whatdoyousee import *
from camera_pi import Camera
from io import BytesIO
from PIL import Image
from time import sleep
from datetime import datetime

vision_api_key = 'a5a3c94b1dee4faa8047873e73e5e642'
vision_api_url = 'https://chinaeast2.api.cognitive.azure.cn/vision/v2.0/analyze'
camera = Camera()

def WhatDoYouSee(body):
    headers = {
        'Content-Type': 'application/octet-stream',
        'Ocp-Apim-Subscription-Key': vision_api_key,
    }
     
    params = {
        'visualFeatures': 'Description, Faces',
        'details': '',
        'language': 'en',
    }

    try:
        api_url = vision_api_url 
        response = requests.post(api_url, headers=headers, data=body, params=params)
        print ('Respose:')
        parsed = json.loads(response.text)
        if len(parsed) == 0:
            parsedText = 'I see nothing'
        else:
            tag_str = ''
            for num_tags in range(5):
                tag_str += parsed['description']['tags'][num_tags] + ', '
            parsedText = 'I see %s. Top 5 tags are: %s' % (parsed['description']['captions'][0]['text'], tag_str)
        print(parsedText)
        print(json.dumps(parsed, sort_keys=True, indent=2))
    except Exception as e:
        print('Error:')
        print(e)
        parsedText = e

    return parsedText

body = camera.get_frame()
parsedText = WhatDoYouSee(body)

Starting camera thread.


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/pi/berryconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/pi/berryconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pi/sherlock-azure/camera.py", line 69, in _thread
    for frame in frames_iterator:
  File "/home/pi/sherlock-azure/camera_pi.py", line 9, in frames
    with picamera.PiCamera() as camera:
  File "/home/pi/berryconda3/lib/python3.6/site-packages/picamera/camera.py", line 433, in __init__
    self._init_preview()
  File "/home/pi/berryconda3/lib/python3.6/site-packages/picamera/camera.py", line 513, in _init_preview
    self, self._camera.outputs[self.CAMERA_PREVIEW_PORT])
  File "/home/pi/berryconda3/lib/python3.6/site-packages/picamera/renderers.py", line 558, in __init__
    self.renderer.inputs[0].connect(source).enable()
  File "/home/pi/berryconda3/lib/python3.6/site-packages/picamer

KeyboardInterrupt: 

In [2]:
# from camera_pi import Camera
import os
from time import sleep

# define the name of the directory to be created

# camera = Camera()
# name = 'John'
# path = "./faces/%s" %(name)
# if not os.path.exists(path):
#     try:
#         os.makedirs(path)
#     except OSError:  
#         print ("Creation of the directory %s failed" % path)
#     else:  
#         print ("Successfully created the directory %s " % path)
# for num in range(20):
#     image_file = './faces/%s/face%d.png' %(name, num)
#     with open(image_file, 'wb') as f:
#         f.write(camera.get_frame())
#         f.close()
#     sleep(0.25)

img = './faces/John.png'
detect_responses = cf.face.detect(img)
face_id_list = []
for faces in detect_responses:
    face_id = faces['faceId']
    print(face_id)
    face_id_list.append(face_id)
identify_responses = cf.face.identify(face_id_list, 'my_friends')
for person in cf.person.lists('my_friends'):
    identified = identify_responses[0]['candidates'][0]
    if person['personId'] == identified['personId']:
        print(person['name'])


388778d4-0226-4f84-a8f0-7a4d9fc4935e
John


In [3]:
responses = cf.person.lists('my_friends')


In [1]:
from datetime import datetime
from camera_pi import Camera
camera = Camera()

image = './faces/Detection_%s.png' %(datetime.now().strftime('%Y%m%d_%H%M%S'))
with open(image, 'wb') as f:
    f.write(camera.get_frame())
    f.close()

Starting camera thread.


In [3]:
detect_responses = cf.face.detect(image)
face_id_list = []
for faces in detect_responses:
    face_id = faces['faceId']
    print(face_id)
    face_id_list.append(face_id)
identify_responses = cf.face.identify(face_id_list, 'my_friends')
print(identify_responses)
for detected in identify_responses:
    for person in cf.person.lists('my_friends'):
        identified = detected['candidates'][0]
        if person['personId'] == identified['personId']:
    #         responses = cf.person.add_face(image, 'my_friends', person['personId'])
    #         responses = cf.person_group.train('my_friends')
            identified_name = person['name']
            print(identified_name)

0525c775-1ca7-4e20-a7b6-9f8862afd46a
[{'candidates': [{'personId': 'b99a95eb-945b-4495-a467-1528f6210e49', 'confidence': 0.6568}], 'faceId': '0525c775-1ca7-4e20-a7b6-9f8862afd46a'}]
John


In [13]:
cf.person.delete('my_friends', 'ce129d49-03e2-4376-b29c-cc4a23cc6bed')

{}

In [4]:
cf.person.lists('my_friends')

[{'personId': '07d26e76-209f-457e-b54c-30cfb6683214',
  'persistedFaceIds': ['2a825319-551e-4247-a0e0-8f71fa245d36',
   '31c17173-37e3-4b2c-801a-2b5a256fc145',
   '4fc08af5-da63-4eb9-aeed-ab940bb4b9e0',
   '626e6f70-05ad-4e13-8311-24921ff62385',
   '644fae01-1b39-4d4a-9e94-8e8da257b14b',
   '6c114d89-5223-4343-9195-3064bb1aefd4',
   '95208953-d850-4b14-acec-0cb782d84335',
   '99fbb52c-f079-4ed6-bfa7-89b42f7f8e73',
   'adc34524-f087-4c2e-9fb6-3b46577f33c5',
   'b3015a76-6ae6-481f-8d1a-cc35bd7d8805',
   'b61bca3c-6623-49f9-967d-b44f9d27607a',
   'ca3dc3c6-892e-432a-9a3f-851f2c731556',
   'ddb307cf-c33a-46c8-af70-500bfac2a694'],
  'name': 'jaeho',
  'userData': None},
 {'personId': '1427b961-17ea-42f1-8197-04b6072588b2',
  'persistedFaceIds': [],
  'name': 'Obama',
  'userData': None},
 {'personId': '51a9bf05-526a-4e65-8a69-019d358ccbf1',
  'persistedFaceIds': ['045ce115-f5ca-4282-bbc5-5dfefd7f572f',
   '1d8e0be8-bab2-4ee9-bbff-abed3b104a82',
   '3df3c991-e684-4c83-86bb-ee114e05875d',
   

In [5]:
from datetime import datetime
print(datetime.now().strftime('%Y%m%d_%H%M%S'))

20190429_012014
